In [69]:
from bs4 import BeautifulSoup   
import requests 
from mylibs import *
import time
from tqdm import tqdm
from typing import List, Tuple
import csv
from joblib import Parallel, delayed
from datetime import datetime
import re
import os
import random
AUTHOR_LIST = []
URL_BASE = "https://quotes.toscrape.com/"
AUTHOR_QUOTES = {}
AUTHOR_CATEGORY = {}

1.1 Crawl Raw Data from the website

In [ ]:
def ScrapRawHTML(max_page: int) -> dict:
    raw_htmls = {}
    for i in range (1, max_page+1):
        url = f"https://quotes.toscrape.com/page/{i}/"  
        respones = requests.get(url)
        soup = BeautifulSoup(respones.text, 'html.parser')
        raw_htmls[f'Page{i}'] = str(soup)
    for key, value in raw_htmls.items():
        final_value = f"======================{key}======================\n{value}\n"
        with open(f"kq.txt", "a", encoding= 'utf-8') as f: #keep writing to the same file without deleting the previous content
            f.write(final_value)         
        
    

In [19]:
ScrapRawHTML(10)

1.2 Perform reuests

In [27]:
def Read_tag_quote():
    with open("kq.txt", "r", encoding= 'utf-8') as f:
        data = f.read()
        pages = re.split(r'={22}Page\d+={22}', data)
        pages.remove('')
        results = []
        for i,page in enumerate(pages):
            soup = BeautifulSoup(page, 'html.parser')
            result = soup.find_all('div', class_='quote')
            results.append({'page': i+1, 'result': str(result)})
        return results
results = Read_tag_quote()
for result in results: 
    print(result)

{'page': 1, 'result': '[<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">\n<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>\n<span>by <small class="author" itemprop="author">Albert Einstein</small>\n<a href="/author/Albert-Einstein">(about)</a>\n</span>\n<div class="tags">\n            Tags:\n            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>\n<a class="tag" href="/tag/change/page/1/">change</a>\n<a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>\n<a class="tag" href="/tag/thinking/page/1/">thinking</a>\n<a class="tag" href="/tag/world/page/1/">world</a>\n</div>\n</div>, <div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">\n<span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>\n<span>by <small

In [28]:
def Read_small_quote():
    with open("kq.txt", "r", encoding= 'utf-8') as f:
        data = f.read()
        pages = re.split(r'={22}Page\d+={22}', data)
        pages.remove('')
        results = []
        for i,page in enumerate(pages):
            soup = BeautifulSoup(page, 'html.parser')
            for quote in soup.find_all('div', class_='quote'):
                author = quote.find('small', class_='author').text
                results.append({'page': i+1, 'result': str(author)})
        return results
results = Read_small_quote()
for result in results: 
    print(result)

{'page': 1, 'result': 'Albert Einstein'}
{'page': 1, 'result': 'J.K. Rowling'}
{'page': 1, 'result': 'Albert Einstein'}
{'page': 1, 'result': 'Jane Austen'}
{'page': 1, 'result': 'Marilyn Monroe'}
{'page': 1, 'result': 'Albert Einstein'}
{'page': 1, 'result': 'André Gide'}
{'page': 1, 'result': 'Thomas A. Edison'}
{'page': 1, 'result': 'Eleanor Roosevelt'}
{'page': 1, 'result': 'Steve Martin'}
{'page': 2, 'result': 'Marilyn Monroe'}
{'page': 2, 'result': 'J.K. Rowling'}
{'page': 2, 'result': 'Albert Einstein'}
{'page': 2, 'result': 'Bob Marley'}
{'page': 2, 'result': 'Dr. Seuss'}
{'page': 2, 'result': 'Douglas Adams'}
{'page': 2, 'result': 'Elie Wiesel'}
{'page': 2, 'result': 'Friedrich Nietzsche'}
{'page': 2, 'result': 'Mark Twain'}
{'page': 2, 'result': 'Allen Saunders'}
{'page': 3, 'result': 'Pablo Neruda'}
{'page': 3, 'result': 'Ralph Waldo Emerson'}
{'page': 3, 'result': 'Mother Teresa'}
{'page': 3, 'result': 'Garrison Keillor'}
{'page': 3, 'result': 'Jim Henson'}
{'page': 3, 'res

In [70]:
# def visited_author():
def get_authors_quotes() -> dict:
   global AUTHOR_QUOTES
   with open("kq.txt", "r", encoding= 'utf-8') as f:
       data = f.read()
       pages = re.split(r'={22}Page\d+={22}', data)
       pages.remove('')
       for i,page in enumerate(pages):
           soup = BeautifulSoup(page, 'html.parser')
           for quote in soup.find_all('div', class_='quote'):
               author = quote.find('small', class_='author').text
               quote = quote.find('span', class_='text').text
               if author in AUTHOR_QUOTES.keys():
                   AUTHOR_QUOTES[author].append(quote)
               else:
                   AUTHOR_QUOTES[author] = [quote] 
   
  
def get_author_category() -> dict:
    global AUTHOR_CATEGORY
    with open("kq.txt", "r", encoding= 'utf-8') as f:
        data = f.read()
        pages = re.split(r'={22}Page\d+={22}', data)
        pages.remove('')
        for i,page in enumerate(pages):
            soup = BeautifulSoup(page, 'html.parser')
            for quote in soup.find_all('div', class_='quote'):
                author = quote.find('small', class_='author').text
                category = quote.find('div', class_='tags').find_all('a')
                category = [cate.text for cate in category]
                if author in AUTHOR_CATEGORY.keys():
                    AUTHOR_CATEGORY[author].extend(category)
                else:
                    AUTHOR_CATEGORY[author] = category
    return AUTHOR_CATEGORY 
def Get_author_info(link :str) -> str:
    get_link = requests.get(link)
    soup = BeautifulSoup(get_link.text, 'html.parser')
    date = soup.find('span', class_='author-born-date').text 
    date =datetime.strptime(date, '%B %d, %Y').strftime('%Y-%m-%d')
    return date
        
def authorLink():
    global AUTHOR_LIST
    get_authors_quotes()
    get_author_category()
    with open("kq.txt", "r", encoding= 'utf-8') as f:
        data = f.read() 
        pages = re.split(r'={22}Page\d+={22}', data)
        pages.remove('')
        authors = []
        for i,page in enumerate(pages):
            soup = BeautifulSoup(page, 'html.parser')
            for quote in soup.find_all('div', class_='quote'):
                author = quote.find('small', class_='author').text
                if author in AUTHOR_LIST:
                    continue
                AUTHOR_LIST.append(author)    
                link = quote.find('a')['href']
                quote = random.choice(AUTHOR_QUOTES[author])
                author_info = {'Author\'s name' : author, 
                               'Author\'s link': URL_BASE + link,
                               'Author\'s date of birth': Get_author_info(URL_BASE + link),
                               'Author\'s famous quote': quote}
                authors.append(author_info) 
               
    return authors

In [71]:
authors_details = authorLink()
print('\n'.join([str(author) for author in authors_details]))

{"Author's name": 'Albert Einstein', "Author's link": 'https://quotes.toscrape.com//author/Albert-Einstein', "Author's date of birth": '1879-03-14', "Author's famous quote": '“Any fool can know. The point is to understand.”'}
{"Author's name": 'J.K. Rowling', "Author's link": 'https://quotes.toscrape.com//author/J-K-Rowling', "Author's date of birth": '1965-07-31', "Author's famous quote": '“It is impossible to live without failing at something, unless you live so cautiously that you might as well not have lived at all - in which case, you fail by default.”'}
{"Author's name": 'Jane Austen', "Author's link": 'https://quotes.toscrape.com//author/Jane-Austen', "Author's date of birth": '1775-12-16', "Author's famous quote": '“I declare after all there is no enjoyment like reading! How much sooner one tires of any thing than of a book! -- When I have a house of my own, I shall be miserable if I have not an excellent library.”'}
{"Author's name": 'Marilyn Monroe', "Author's link": 'https:/

In [76]:
def writeto_csv(authors,file_name = 'Quote.csv',type = None):
    with open(file_name, 'w', encoding='utf-8', newline='') as file:
        if isinstance(authors, list):  
            fieldnames = authors[0].keys()
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(authors)
        elif isinstance(authors, dict):  
            fieldnames = ["Author's name",type]
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for author, quotes in authors.items():
                writer.writerow({
                    "Author's name": author,
                    type : quotes if isinstance(quotes, str) else ' | '.join(quotes)
                })
writeto_csv(authors_details)
writeto_csv(AUTHOR_QUOTES, 'Quotes_Of_Authors.csv', 'Author\'s famous quotes')
writeto_csv(AUTHOR_CATEGORY, 'Category_Of_Authors.csv', 'Author\'s category')
